<a href="https://colab.research.google.com/github/jfb10d/ml-practice/blob/main/notebooks/fcc_predict_health_costs_with_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
dataset.shape

In [ ]:
dataset.isnull().sum()

In [ ]:
X = dataset.drop('expenses', axis=1)
y = dataset['expenses']

In [ ]:
from sklearn.model_selection import train_test_split

# X: Features (independent variables)
# y: Target (dependent variable)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.columns = X.columns
X_test.columns = X.columns

In [ ]:
categorical_features = ['sex', 'smoker', 'region']
numerical_features = ['age', 'bmi', 'children']

In [ ]:
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
ordinal_transformer = OrdinalEncoder()

preprocessor = ColumnTransformer(
    [
        ("OrdinalEncoder", ordinal_transformer, categorical_features),
         ("StandardScaler", numeric_transformer, numerical_features),
    ]
)
X_tr = preprocessor.fit(X_train).transform(X_train)
X_te = preprocessor.fit(X_train).transform(X_test)

In [ ]:
model = tf.keras.Sequential([
    layers.Dense(units=96, input_shape=(X_tr.shape[1],)),
    layers.Dense(units=1)
])
model.summary()

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.05),
    loss='mean_absolute_error',
    metrics=['mean_absolute_error', 'mean_squared_error'])

In [ ]:
callback = keras.callbacks.EarlyStopping(monitor='val_loss',
                                              patience=10, min_delta=10)

history = model.fit(
    X_tr,
    y_train,
    epochs=200,
    # Suppress logging.
    batch_size=144,
    verbose=1,
    # Calculate validation results on 20% of the training data.
    validation_split = 0.2,
    callbacks=[callback])

In [ ]:
test_dataset = X_te
test_labels = y_test

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
